In [1]:
from linear_depth_circuit import *
from matrix_simulation import *
from naive_circuit import *

In [112]:
# Number of position qubits
n = 7
# Number of steps
n_step = 1
# Number of times we want to run the circuits
n_shot = 1000
# Angles for the coin operators
angles = random_angles(n)

# Linear-depth position-dependent coin operators quantum circuit
#linear_qc = quantum_walk_linear_depth_circuit(n,angles,n_step)
linear_qc = build_linear_depth_circuit(n,angles)
#counts1 = simulate_circuit(linear_qc,n_shot)

# Naive position-dependent coin operators quantum circuit
#naive_qc = quantum_walk_naive_circuit(n,angles,n_step)
naive_qc = build_naive_circuit(n,angles)
#counts2 = simulate_circuit(naive_qc,n_shot)

# Matrix simulation
v = position_state(n)
c = coin_state()
q = quantum_state(v,c)

#q_final = quantum_walk_simulation(n,q,angles,n_step)

#counts3 = results(n,q_final)
# Convert the matrix simulation results in qiskit dictionary
#counts3 = Counts(counts3)

#plot_histogram([counts1,counts2, counts3], legend=['Linear-depth','Naive', 'Matrix simulation'], color=['blue','magenta','orange'])

In [113]:
linear_qc.depth()

55

In [114]:
naive_qc.depth()

256

In [115]:
print(dict(linear_qc.count_ops()))

{'cx': 960, 'cswap': 508, 'cu': 128, 'x': 2}


In [116]:
print(dict(naive_qc.count_ops()))

{'x': 254, 'c7u': 128}


In [117]:
simulator = QasmSimulator()
compiled_circuit = transpile(linear_qc, simulator)

In [118]:
compiled_circuit.depth()

55

In [119]:
print(dict(compiled_circuit.count_ops()))

{'cx': 864, 'cswap': 508, 'cu': 128, 'x': 2}


In [120]:
simulator = QasmSimulator()
compiled_circuit = transpile(naive_qc, simulator)

In [121]:
compiled_circuit.depth()

227200

In [122]:
print(dict(compiled_circuit.count_ops()))

{'cx': 145920, 'u': 48768, 'p': 32511, 'u1': 32383, 'u3': 16258, 'x': 252}
